# **Minimax Search – Enhanced Jupyter Notebook (2025 Edition)**

This notebook gives you a **complete, exam-ready + implementation-focused** understanding of **Minimax Search** using a **Tic-Tac-Toe game** as the running example.

### ✅ You will learn:
- What is adversarial search?
- Game tree, MAX / MIN levels
- Minimax algorithm (top-down intuition)
- Full Python implementation
- Tic-Tac-Toe AI using Minimax
- Step-by-step **visualization & animations** in the notebook
- How to instrument the algorithm (count nodes, display recursion tree)
- Time complexity & practical limitations


## **1. Adversarial Search & Game Trees**

We are dealing with **two-player, turn-based, zero-sum, perfect-information games**, e.g.:
- Tic-Tac-Toe
- Chess (simplified)
- Checkers
- Connect-4

### Key Terms
- **State**: Current configuration of the board.
- **Player**: One of {MAX, MIN}. We usually assume:
  - MAX → Our AI (tries to **maximize** the score)
  - MIN → Opponent (tries to **minimize** the score)
- **Move**: Transition from one state to another.
- **Terminal State**: Game over (win, lose, draw).
- **Utility / Payoff**:
  - +1 → MAX wins
  - -1 → MIN wins
  - 0 → Draw

### Minimax Idea (Tree View)
We build a game tree and evaluate outcome assuming:
- MAX plays **optimally**.
- MIN also plays **optimally**.


## **2. Minimax Algorithm – Pseudocode**

For a node `s` with player `P(s)`:

```text
MINIMAX(s):
  if s is terminal:
      return utility(s)

  if P(s) == MAX:
      best = -∞
      for each child c of s:
          val = MINIMAX(c)
          best = max(best, val)
      return best
  else:
      best = +∞
      for each child c of s:
          val = MINIMAX(c)
          best = min(best, val)
      return best
```

We will now **implement this for Tic-Tac-Toe**.


## **3. Tic-Tac-Toe Board Representation**

We use:
- 3×3 grid
- `'X'` → MAX (AI)
- `'O'` → MIN (Human or second AI)
- `' '` → Empty cell

Let's define helper functions:
- Print board
- Check winner
- Get list of possible moves
- Check terminal state


In [ ]:
from typing import List, Optional, Tuple

Board = List[List[str]]

def create_empty_board() -> Board:
    return [[' ' for _ in range(3)] for _ in range(3)]

def print_board(board: Board) -> None:
    """Pretty print Tic-Tac-Toe board."""
    for i, row in enumerate(board):
        print(' | '.join(row))
        if i < 2:
            print('-' * 5)
    print()

def check_winner(board: Board) -> Optional[str]:
    """Return 'X', 'O' or None based on current board."""
    lines = []

    # Rows and columns
    for i in range(3):
        lines.append(board[i])  # row i
        lines.append([board[0][i], board[1][i], board[2][i]])  # column i

    # Diagonals
    lines.append([board[0][0], board[1][1], board[2][2]])
    lines.append([board[0][2], board[1][1], board[2][0]])

    for line in lines:
        if line[0] != ' ' and line.count(line[0]) == 3:
            return line[0]

    return None

def is_full(board: Board) -> bool:
    return all(board[i][j] != ' ' for i in range(3) for j in range(3))

def is_terminal(board: Board) -> bool:
    return check_winner(board) is not None or is_full(board)

def get_empty_positions(board: Board) -> List[Tuple[int,int]]:
    cells = []
    for i in range(3):
        for j in range(3):
            if board[i][j] == ' ':
                cells.append((i,j))
    return cells

# Quick test
b = create_empty_board()
b[0][0] = 'X'
b[1][1] = 'O'
print_board(b)
print("Winner:", check_winner(b))


## **4. Minimax Implementation (Without Alpha-Beta)**

We define a scoring function:
- `+1` → 'X' (MAX) wins
- `-1` → 'O' (MIN) wins
- `0` → draw

We implement `minimax(board, is_max_turn)` which returns:
- **Score** of the position
- **Best move** for the current player


In [ ]:
def evaluate(board: Board) -> int:
    winner = check_winner(board)
    if winner == 'X':
        return 1
    elif winner == 'O':
        return -1
    else:
        return 0

def minimax(board: Board, is_max_turn: bool):
    """Return (best_score, best_move) using minimax."""
    if is_terminal(board):
        return evaluate(board), None

    if is_max_turn:
        best_score = float('-inf')
        best_move = None
        for (i,j) in get_empty_positions(board):
            board[i][j] = 'X'
            score, _ = minimax(board, False)
            board[i][j] = ' '
            if score > best_score:
                best_score = score
                best_move = (i,j)
        return best_score, best_move
    else:
        best_score = float('inf')
        best_move = None
        for (i,j) in get_empty_positions(board):
            board[i][j] = 'O'
            score, _ = minimax(board, True)
            board[i][j] = ' '
            if score < best_score:
                best_score = score
                best_move = (i,j)
        return best_score, best_move

# Test minimax from starting position
board = create_empty_board()
score, move = minimax(board, True)
print("Best score from start:", score)
print("Best first move for X:", move)


## **5. Visualizing Minimax Decisions (Node Expansion Count)**

To understand how many states are explored, we instrument the algorithm with a **global node counter**.


In [ ]:
node_count = 0

def minimax_with_count(board: Board, is_max_turn: bool):
    global node_count
    node_count += 1

    if is_terminal(board):
        return evaluate(board), None

    if is_max_turn:
        best_score = float('-inf')
        best_move = None
        for (i,j) in get_empty_positions(board):
            board[i][j] = 'X'
            score, _ = minimax_with_count(board, False)
            board[i][j] = ' '
            if score > best_score:
                best_score = score
                best_move = (i,j)
        return best_score, best_move
    else:
        best_score = float('inf')
        best_move = None
        for (i,j) in get_empty_positions(board):
            board[i][j] = 'O'
            score, _ = minimax_with_count(board, True)
            board[i][j] = ' '
            if score < best_score:
                best_score = score
                best_move = (i,j)
        return best_score, best_move

board = create_empty_board()
node_count = 0
score, move = minimax_with_count(board, True)
print("Best move:", move, "score:", score)
print("Total nodes explored:", node_count)


## **6. Animation: AI vs AI (Minimax vs Minimax)**

We now create a **simple animation** where:
- X and O are both controlled by minimax.
- We use `IPython.display.clear_output` to **animate** the board in the output cell.


In [ ]:
from IPython.display import clear_output
import time

def minimax_best_move(board: Board, is_max_turn: bool):
    score, move = minimax(board, is_max_turn)
    return move

def play_ai_vs_ai(delay: float = 0.7):
    board = create_empty_board()
    is_max_turn = True

    while not is_terminal(board):
        clear_output(wait=True)
        print("Current player:", 'X' if is_max_turn else 'O')
        print_board(board)
        move = minimax_best_move(board, is_max_turn)
        if move is None:
            break
        i, j = move
        board[i][j] = 'X' if is_max_turn else 'O'
        is_max_turn = not is_max_turn
        time.sleep(delay)

    clear_output(wait=True)
    print("Final Board:")
    print_board(board)
    winner = check_winner(board)
    if winner:
        print("Winner:", winner)
    else:
        print("Result: Draw")


# Run this cell to see an animated AI vs AI game.
# play_ai_vs_ai()


> 💡 **Note:** The animation will run only when you **uncomment** the line `play_ai_vs_ai()` and execute the cell in Jupyter.

---

## **7. Time & Space Complexity of Minimax**

Let:
- `b` = branching factor (number of possible moves at each state)
- `d` = maximum depth of tree (till terminal state)

Then:
- **Time complexity**: `O(b^d)`
- **Space complexity**: `O(b·d)` (for recursion stack)

For Tic-Tac-Toe:
- b is at most 9, depth at most 9 → still manageable.

For Chess-like games:
- Tree grows huge → need **Alpha-Beta Pruning**, heuristic evaluation, depth limiting.


## **8. Practice Tasks (Minimax Only)**

1. Modify minimax to return the *second-best move* also.
2. Limit minimax depth to `k` and use a **heuristic evaluation** instead of exact win/loss at leaves.
3. Implement Connect-4 with minimax (small board version).
4. Add randomization: if two moves have equal score, pick one randomly.
5. Print the **full game tree** (state + score) for a partially filled Tic-Tac-Toe board.
